In [ ]:
%load_ext autoreload
%autoreload 2
import numpy as np
import matplotlib.pyplot as plt
optimization_config = {
    'eval_budget': 50
}
import os
from fig_hp import *
os.environ['KMP_DUPLICATE_LIB_OK'] = 'True'
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

# plt.rcParams.update({
#     "text.usetex": True,
#     "font.family": "sans-serif",
#     "font.sans-serif": ["Helvetica"]})
# for Palatino and other serif fonts use:
plt.rcParams.update({
    "text.usetex": True,
    "font.family": "serif",
    "font.serif": ["Palatino"],
})
OURALG = 'CONFIG'
#COLORS = ['green','red', 'yellow', 'orange', 'blue', 'black']
#COLORS = [(0.0, 0.63, 0.53),(0.90,0.29,0.21), (0.87, 0.56, 0.27), (0.30, 0.73, 0.84), (0.39, 0.09, 0.47), (0.22, 0.31, 0.33)]
COLORS = ['#008280FF', '#BB0021FF', '#808180FF', '#3B4992FF', '#631879FF', '#5F559BFF']
HATCHES = ['', '', '', '','','']
from matplotlib import rcParams
#rcParams['font.family'] = 'serif'
#rcParams['font.serif'] = ['Times']


import matplotlib as mpl
mpl.use('pdf')
FONTDICT={'fontsize':LABELSIZE}

params = {
   'axes.labelsize': LABELSIZE,
   #'text.fontsize': LABELSIZE,
   'legend.fontsize': LABELSIZE,
   'xtick.labelsize': LABELSIZE,
   'ytick.labelsize': LABELSIZE,
   'text.usetex': False,
    'hatch.linewidth': 0.5,
   'figure.figsize': [WIDTH, HEIGHT]
   }
rcParams.update(params)
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
#jtplot.style()
from datetime import datetime
import time
%load_ext autoreload
%autoreload 2

In [ ]:
import ApartmentsThermal
start_time = time.time()
print(ApartmentsThermal.get_ApartTherm_kpis(controller='PH', params=(0.8, 18, 6)))
end_time = time.time()

In [ ]:
(end_time - start_time) / 60

In [ ]:
# analyze the gridding data
num_grid = 5
lower_grids = [0.1, 0.15, 0.2, 0.25, 0.3] # , 0.5, 0.7, 0.9]
upper_grids = [18, 18.5, 19, 19.5, 20.0] #, 22, 24, 25]
energy_array = np.zeros((num_grid, num_grid))
dev_array = np.zeros((num_grid, num_grid))

for i in range(5):
    lower_thr = lower_grids[i]
    for j in range(5):
        upper_thr = upper_grids[j]
        energy, dev = ApartmentsThermal.get_ApartTherm_kpis(controller='P', params=(lower_thr, upper_thr))
        
        energy_array[i, j] = energy
        dev_array[i, j] = dev 


In [ ]:
%matplotlib inline
plt.scatter(energy_array.reshape(-1), dev_array.reshape(-1))
plt.show()

plt.imshow(energy_array, cmap='hot', interpolation='nearest')
plt.show()

plt.imshow(dev_array, cmap='hot', interpolation='nearest')
plt.show()

print(energy_array)
print(dev_array)

# analyze grid data

In [ ]:
data = np.load('./data/apart_therm_grid_data.npz', allow_pickle=True)

In [ ]:
%matplotlib inline
energy_grid_data = energy_array #data['arr_0']
dev_grid_data = dev_array #data['arr_1']

# get stats of energy and dev
print(np.mean(energy_grid_data), np.std(energy_grid_data))
print(np.mean(dev_grid_data), np.std(dev_grid_data))

plt.imshow(energy_grid_data, cmap='hot', interpolation='nearest')
plt.show()

plt.imshow(dev_grid_data, cmap='hot', interpolation='nearest')
plt.show()

plt.scatter(energy_grid_data.reshape(-1), dev_grid_data.reshape(-1))

print(dev_grid_data)

# BO on the tuning problem

In [ ]:
from test_sample_Gaussian import *

In [ ]:
import simple_util
from test_ApartTherm_util import *
problem_name = 'energym_apartment_therm_tune_PH'
problem_config = simple_util.get_config(problem_name)

safe_bo_config = {
    'noise_level': 0.0,
    'kernel_var': 0.1,
    'train_noise_level': 0.0,
    'problem_name': 'energym_apartment_therm_tune_PH'
}

lcb2_config = safe_bo_config.copy()
lcb2_config.update({
        'total_eval_num': optimization_config['eval_budget'],
        }
    )
lcb2_opt, lcb2_best_obj_list, lcb2_total_cost_list = get_optimizer(
            'lcb2', lcb2_config, problem_config)
lcb2_opt_obj_list = lcb2_opt.init_obj_val_list
lcb2_opt_constr_list = lcb2_opt.init_constr_val_list

#lcb2_costs, lcb2_objs, lcb2_opt, lcb2_obj_traj, lcb2_constrs = get_lcb2_result(problem_config)
#safe_costs, safe_objs, safe_opt, safe_obj_traj, safe_constrs = get_safe_bo_result(problem_config, plot=False)
#con_costs, con_objs, con_opt, con_obj_traj, con_constrs = get_constrained_bo_result(problem_config, plot=False)
#pdbo_costs, pdbo_objs, pdbo_opt, pdbo_obj_traj, pdbo_constrs = get_pdbo_result(problem_config)
   

In [ ]:
#for _ in range(optimization_config['eval_budget']):
    #if True:
    #if True:
if True:
    print(f'lcb2 step {_}.')
    y_obj, constr_vals = lcb2_opt.make_step()
    lcb2_total_cost_list.append(lcb2_opt.cumu_vio_cost)
    lcb2_best_obj_list.append(lcb2_opt.best_obj)
    lcb2_opt_obj_list.append(y_obj)
    lcb2_opt_constr_list.append(constr_vals)


In [ ]:
lcb2_opt.gp_obj.X

In [ ]:
problem_config['obj'](np.array([[0.2, 0.8]]))

# analyze the result

In [ ]:
# read the data
data = np.load('test_ApartTherm_result_with_pdbo_PH_16_47_34-Dec_22_2022.npz', allow_pickle=True)
epbo_data = np.load('test_ApartTherm_result_with_only_epbo_PH_0.2_1.0_3.0_15_26_56-Dec_22_2022.npz', allow_pickle=True)

In [ ]:
def fix_first_ele(regret_list):
    num_traj, num_steps = regret_list.shape
    for k in range(num_traj):
        regret_list[k, 0] = np.array([[regret_list[k,0]]])
    return regret_list


safe_ave_cost_arr = data['arr_0'] 
safe_ave_simple_regret_arr = data['arr_1']
con_ave_cost_arr = data['arr_2']
con_ave_simple_regret_arr = data['arr_3']  
vabo_ave_cost_arr_set = data['arr_4']
vabo_ave_simple_regret_arr_set = data['arr_5']

safe_cost_lists = data['arr_6']
safe_simple_regret_lists = data['arr_7']
con_bo_cost_lists = data['arr_8']
con_bo_simple_regret_lists = data['arr_9'] 
vabo_cost_lists = data['arr_10']
vabo_simple_regret_lists = data['arr_11']

safe_regret_lists = fix_first_ele(data['arr_14'])
safe_constrs_lists = data['arr_15']
con_bo_regret_lists = fix_first_ele(data['arr_16'])

con_bo_constrs_lists = data['arr_17'] 
pdbo_regret_lists = fix_first_ele(data['arr_18']) 
pdbo_constrs_lists = data['arr_19']
vabo_regret_lists_set = data['arr_20']
vabo_constrs_lists_set = data['arr_21']

lcb2_regret_lists = fix_first_ele(data['arr_22'])
lcb2_constrs_lists = data['arr_23']


epbo_regret_lists_0 = fix_first_ele(epbo_data['arr_24'])
epbo_constrs_lists_0 = epbo_data['arr_25']

epbo_regret_lists_1 = fix_first_ele(epbo_data['arr_26'])
epbo_constrs_lists_1 = epbo_data['arr_27']

epbo_regret_lists_2 = fix_first_ele(epbo_data['arr_28'])
epbo_constrs_lists_2 = epbo_data['arr_29']






In [ ]:
len(data['arr_14'][0])

In [ ]:
# compare cost of different methods
EPSILON=1e-4
def plot_results():
    plt.figure()
    plt.plot(np.array(safe_bo_total_cost_list)[:,  0], marker='*')
    plt.plot(np.array(constrained_bo_total_cost_list)[:, 0], marker='o')
    legends_list = ['Safe BO', 'Generic Constrained BO']

    for violation_aware_bo_total_cost_list, budget in violation_aware_bo_total_cost_list_set:
        plt.plot(np.array(violation_aware_bo_total_cost_list)[:, 0], marker='+')
        legends_list.append('Violation Aware BO '+str(budget))
        #print(violation_aware_bo_total_cost_list)

    #plt.plot(np.array([vio_budget]*(optimization_config['eval_budget']+1)), marker='v', color='r')
    plt.xlim((0, optimization_config['eval_budget']+1))
    #plt.ylim((0, 500))
    print(legends_list)
    plt.legend(legends_list)
    plt.xlabel('Step')
    plt.ylabel('Cumulative cost')
    plt.savefig('./fig/cost_inc.png', format='png')
    # compare convergence
    plt.figure()
    plt.plot(safe_bo_best_obj_list, marker='*')
    plt.plot(constrained_bo_best_obj_list, marker='o')

    for violation_aware_bo_best_obj_list, budget in violation_aware_bo_best_obj_list_set:
        plt.plot(violation_aware_bo_best_obj_list, marker='+')

    plt.xlim((0, optimization_config['eval_budget']+1))
    #plt.ylim((0, 500))
    #plt.xlim((0, 20))
    plt.legend(legends_list)
    plt.xlabel('step')
    plt.ylabel('Simple Regret')
    plt.savefig('./fig/best_obj.png', format='png')


In [ ]:
instance_id = 0

plt.plot(lcb2_regret_lists[instance_id, :])
plt.plot(lcb2_constrs_lists[instance_id, :])
plt.legend(['Regret', 'Constraint'])

In [ ]:
%matplotlib inline
_, total_steps = pdbo_regret_lists.shape
plt.figure()
plt.plot(np.cumsum(np.mean(np.squeeze(con_bo_regret_lists), axis=0))/np.arange(1, total_steps+1))
plt.plot(np.cumsum(np.mean(np.squeeze(safe_regret_lists), axis=0))/np.arange(1, total_steps+1))
plt.plot(np.cumsum(np.mean(np.squeeze(pdbo_regret_lists), axis=0))/np.arange(1, total_steps+1))
plt.plot(np.cumsum(np.mean(np.squeeze(lcb2_regret_lists), axis=0))/np.arange(1, total_steps+1))
plt.xlabel('Steps')
plt.ylabel('Average Regret')
#plt.ylim((0, 0.4))
plt.legend(['CEI', 'Safe BO', 'Primal-Dual', 'LCB2'])
#plt.savefig('./fig/sample_SE_ave_regret.pdf', format='pdf')
plt.show()

plt.figure()
plt.plot(np.cumsum(np.mean(np.maximum(np.squeeze(con_bo_constrs_lists), 0), axis=0))/np.arange(1, total_steps+1))
plt.plot(np.cumsum(np.mean(np.maximum(-np.squeeze(safe_constrs_lists), 0), axis=0))/np.arange(1, total_steps+1))
plt.plot(np.cumsum(np.mean(np.maximum(np.squeeze(pdbo_constrs_lists), 0), axis=0))/np.arange(1, total_steps+1))
plt.plot(np.cumsum(np.mean(np.maximum(np.squeeze(lcb2_constrs_lists), 0), axis=0))/np.arange(1, total_steps+1))
plt.xlabel('Steps')
plt.ylabel('Average Violation')
plt.legend(['CEI', 'Safe BO', 'Primal-Dual', 'LCB2'])
#plt.savefig('./fig/sample_SE_ave_vio.pdf', format='pdf')
plt.show()



In [ ]:
energy_mean = 1500.954861111111
energy_std =  12.634541238981747
dev_mean = 0.31793574850163836
dev_std =  0.03637075568519829



# draw cumulative regret

In [ ]:
MARKERSIZE = 1.5 * 0.4 * 0.1
colors = ['red','green','purple', 'orange', 'blue', 'black', 'blue']
marksCur = ['o','s', '*', 'v', '^','+']
def plot_with_shade_area(ax, x, y, yerr, **kwargs):
    aver_line = ax.plot(x, y,markersize=MARKERSIZE, linewidth=LINEWIDTH, color=kwargs['color'], marker=kwargs['marker'], markerfacecolor='None')
    aver_region = None #ax.fill_between(x, y-yerr, y+yerr, alpha=0.3,color=kwargs['color'])
    return aver_line, aver_region

def plot_cumu_result(con_bo_regret_lists, safe_regret_lists, pdbo_regret_lists, lcb2_regret_lists,
                    con_bo_constrs_lists, safe_constrs_lists, pdbo_constrs_lists, lcb2_constrs_lists):
    num_traj, total_steps = pdbo_regret_lists.shape
    if False:
    #for i in range(num_traj):
        for j in range(total_steps):
            con_bo_regret_lists[i,j] = con_bo_regret_lists[i,j][0,0]
            safe_regret_lists[i,j] = safe_regret_lists[i,j][0,0]
            pdbo_regret_lists[i,j] = pdbo_regret_lists[i,j][0,0]
            lcb2_regret_lists[i,j] = lcb2_regret_lists[i,j][0,0]
            con_bo_constrs_lists[i,j] = con_bo_constrs_lists[i,j][0]
            safe_constrs_lists[i,j] = safe_constrs_lists[i,j][0]
            pdbo_constrs_lists[i,j] = pdbo_constrs_lists[i,j][0]
            lcb2_constrs_lists[i,j] = lcb2_constrs_lists[i,j][0]
    
    con_bo_regret_lists = con_bo_regret_lists.astype(float)
    safe_regret_lists = safe_regret_lists.astype(float)
    pdbo_regret_lists = pdbo_regret_lists.astype(float)
    lcb2_regret_lists = lcb2_regret_lists.astype(float)
    con_bo_constrs_lists = con_bo_constrs_lists.astype(float)
    safe_constrs_lists = safe_constrs_lists.astype(float)
    pdbo_constrs_lists = pdbo_constrs_lists.astype(float)
    lcb2_constrs_lists = lcb2_constrs_lists.astype(float)
    step_arr = np.arange(total_steps)
    min_energy = np.min([np.min(con_bo_regret_lists*energy_std+energy_mean),
                         np.min(safe_regret_lists*energy_std+energy_mean),
                         np.min(pdbo_regret_lists*energy_std+energy_mean),
                         np.min(lcb2_regret_lists*energy_std+energy_mean)
                        ])
    con_bo_cumu_sum = np.cumsum(np.mean((con_bo_regret_lists*energy_std+energy_mean)-min_energy, axis=0))#/np.arange(1, total_steps+1)
    safe_bo_cumu_sum = np.cumsum(np.mean((safe_regret_lists*energy_std+energy_mean)-min_energy, axis=0))#/np.arange(1, total_steps+1)
    pdbo_cumu_sum = np.cumsum(np.mean((pdbo_regret_lists*energy_std+energy_mean)-min_energy, axis=0))#/np.arange(1, total_steps+1)
    lcb2_cumu_sum = np.cumsum(np.mean((lcb2_regret_lists*energy_std+energy_mean)-min_energy, axis=0))#/np.arange(1, total_steps+1)
    #print(con_bo_regret_lists)
    con_bo_cumu_sd = np.std(np.cumsum((con_bo_regret_lists), axis=1), axis=0)
    safe_bo_cumu_sd = np.std(np.cumsum((safe_regret_lists), axis=1), axis=0)
    pdbo_cumu_sd = np.std(np.cumsum((pdbo_regret_lists), axis=1), axis=0)
    lcb2_cumu_sd = np.std(np.cumsum((lcb2_regret_lists), axis=1), axis=0)
    print(con_bo_cumu_sum, safe_bo_cumu_sum)
    
    con_bo_vio = np.cumsum(np.mean(np.maximum((con_bo_constrs_lists), 0)*dev_std, axis=0))
    safe_bo_vio = np.cumsum(np.mean(np.maximum(-(safe_constrs_lists), 0)*dev_std, axis=0))
    pdbo_vio = np.cumsum(np.mean(np.maximum((pdbo_constrs_lists), 0)*dev_std, axis=0))
    lcb2_vio = np.cumsum(np.mean(np.maximum((lcb2_constrs_lists), 0)*dev_std, axis=0))
    
    con_bo_vio_sd = np.std(np.cumsum(np.maximum((con_bo_constrs_lists),0), axis=1), axis=0)
    safe_bo_vio_sd = np.std(np.cumsum(np.maximum((safe_constrs_lists), 0), axis=1), axis=0)
    pdbo_vio_sd = np.std(np.cumsum(np.maximum((pdbo_constrs_lists),0), axis=1), axis=0)
    lcb2_vio_sd = np.std(np.cumsum(np.maximum((lcb2_constrs_lists),0), axis=1), axis=0)
    
    %matplotlib inline
    #from matplotlib import rc
    #plt.rc('text',usetex=False)
   

    
    fig, axes = plt.subplots(2, 1)
    fig.set_size_inches(WIDTH * 1.0, HEIGHT * 2.4)

    fig.subplots_adjust(left=.22, bottom=.20, right=.99, top=.97, hspace=0.5, wspace=0.4) 
   
    ax = axes[0]
    BETA1 = 0.2
    plot_with_shade_area(ax, step_arr, con_bo_cumu_sum, BETA1*con_bo_cumu_sd,
                         linewidth=LINEWIDTH, color=colors[0], marker=marksCur[0], markerfacecolor='None', markersize=MARKERSIZE)
    plot_with_shade_area(ax, step_arr, safe_bo_cumu_sum, BETA1*safe_bo_cumu_sd,
                         linewidth=LINEWIDTH, color=colors[1], marker=marksCur[1], markerfacecolor='None', markersize=MARKERSIZE)
    plot_with_shade_area(ax, step_arr, pdbo_cumu_sum, BETA1*pdbo_cumu_sd, 
                         linewidth=LINEWIDTH, color=colors[2], marker=marksCur[2], markerfacecolor='None', markersize=MARKERSIZE)
    plot_with_shade_area(ax, step_arr, lcb2_cumu_sum, BETA1*lcb2_cumu_sd,
                         linewidth=LINEWIDTH, color=colors[3], marker=marksCur[3], markerfacecolor='None', markersize=MARKERSIZE)

    plt.xticks(fontsize=LABELSIZE)
    plt.yticks(fontsize=LABELSIZE)
    ax.set_xlabel('Step', fontsize=LABELSIZE)
    ax.set_ylabel('Cumulative Energy', fontsize=LABELSIZE)  

    #ax.legend(['CEI', 'Safe BO', 'Primal-Dual', 'LCB2'])
    
    ax = axes[1]

    # phi_line = ax.plot(con_bo_vio, linewidth=LINEWIDTH, color=colors[-1], marker=marksCur[1], markerfacecolor='None', markersize=MARKERSIZE)
    BETA2 = 0.2
    plot_with_shade_area(ax, step_arr, con_bo_vio, BETA2*con_bo_vio_sd, 
                         linewidth=LINEWIDTH, color=colors[0], marker=marksCur[0], markerfacecolor='None')
    plot_with_shade_area(ax, step_arr, safe_bo_vio, BETA2*safe_bo_vio_sd,
                        linewidth=LINEWIDTH, color=colors[1], marker=marksCur[1], markerfacecolor='None')
    plot_with_shade_area(ax, step_arr, pdbo_vio, BETA2*pdbo_vio_sd,
                        linewidth=LINEWIDTH, color=colors[2], marker=marksCur[2], markerfacecolor='None')
    plot_with_shade_area(ax, step_arr, lcb2_vio, BETA2*lcb2_vio_sd,
                        linewidth=LINEWIDTH, color=colors[3], marker=marksCur[3], markerfacecolor='None')
   
    ax.legend(['CEI', 'Safe BO', 'Primal-Dual', 'LCB2'], loc='lower center', bbox_to_anchor=(0.29, -0.6, 0.5, 0.5),
             ncol=4)

    plt.xticks(fontsize=LABELSIZE)
    plt.yticks(fontsize=LABELSIZE)
    ax.set_xlabel('Step', fontsize=LABELSIZE)
    ax.set_ylabel('Cumulative Deviation', fontsize=LABELSIZE)
    #ax.set_xscale('log')
    #ax.set_title('The Second Contextual Variable', fontsize=LABELSIZE)
    #ax.set_ylim(0, 0.2)
    #lgd = plt.legend(['', ''], loc='upper right', bbox_to_anchor=(1.0, 1.0), ncol=1, prop={'size': LABELSIZE})
    #fig.set_size_inches(WIDTH, HEIGHT)

    #lgd = fig.legend((aver_line[0], adv_line[0]), ('Average', 'Adversarial'), 
    #                 loc='upper right', bbox_to_anchor=(0.78, 0.1), ncol=3, prop={'size': LABELSIZE})


    #plt.savefig('./fig/ave_adv_comp.pdf', format='pdf', bbox_extra_artists=(lgd,), bbox_inches='tight')
    plt.savefig(f'./fig/building_funcs_cumu.pdf', format='pdf', bbox_inches='tight')
    plt.show()  
    
plot_cumu_result(con_bo_regret_lists, safe_regret_lists, pdbo_regret_lists, lcb2_regret_lists,
                    con_bo_constrs_lists, safe_constrs_lists, pdbo_constrs_lists, lcb2_constrs_lists)


In [ ]:
plt.scatter(lcb2_regret_lists, lcb2_constrs_lists)

# draw approximately feasible optimal solution

In [ ]:
from fig_hp import *

In [ ]:
energy_std

In [ ]:

def plot_with_shade_area(ax, x, y, yerr, **kwargs):
    aver_line = ax.plot(x, y,markersize=MARKERSIZE, linewidth=LINEWIDTH, color=kwargs['color'], marker=kwargs['marker'], markerfacecolor='None')
    aver_region = None #ax.fill_between(x, y-yerr, y+yerr, alpha=0.3,color=kwargs['color'])
    return aver_line, aver_region

def plot_best_result(con_bo_regret_lists, safe_regret_lists, pdbo_regret_lists, lcb2_regret_lists,
                    con_bo_constrs_lists, safe_constrs_lists, pdbo_constrs_lists, lcb2_constrs_lists):
    num_traj, total_steps = pdbo_regret_lists.shape
    if False:
    #for i in range(num_traj):
        for j in range(total_steps):
            con_bo_regret_lists[i,j] = con_bo_regret_lists[i,j][0,0]
            safe_regret_lists[i,j] = safe_regret_lists[i,j][0,0]
            pdbo_regret_lists[i,j] = pdbo_regret_lists[i,j][0,0]
            lcb2_regret_lists[i,j] = lcb2_regret_lists[i,j][0,0]
            con_bo_constrs_lists[i,j] = con_bo_constrs_lists[i,j][0]
            safe_constrs_lists[i,j] = safe_constrs_lists[i,j][0]
            pdbo_constrs_lists[i,j] = pdbo_constrs_lists[i,j][0]
            lcb2_constrs_lists[i,j] = lcb2_constrs_lists[i,j][0]
    
    con_bo_regret_lists = con_bo_regret_lists.astype(float)
    safe_regret_lists = safe_regret_lists.astype(float)
    pdbo_regret_lists = pdbo_regret_lists.astype(float)
    lcb2_regret_lists = lcb2_regret_lists.astype(float)
    
    con_bo_constrs_lists = con_bo_constrs_lists.astype(float)
    safe_constrs_lists = safe_constrs_lists.astype(float)
    pdbo_constrs_lists = pdbo_constrs_lists.astype(float)
    lcb2_constrs_lists = lcb2_constrs_lists.astype(float)
    step_arr = np.arange(total_steps)
    energy_mean = 808 #1117.9042996350206
    energy_std = 311 #112.68479944833072
    shift = energy_mean/energy_std#+dev_mean/dev_std
    con_bo_best_regret_constr_lists = np.minimum.accumulate(np.squeeze(con_bo_regret_lists) + np.maximum(np.squeeze(con_bo_constrs_lists), 0)+shift, axis=0)
    safe_best_regret_constr_lists = np.minimum.accumulate(np.squeeze(safe_regret_lists) + np.maximum(np.squeeze(safe_constrs_lists), 0)+shift, axis=0)
    pdbo_best_regret_constr_lists = np.minimum.accumulate(np.squeeze(pdbo_regret_lists) + np.maximum(np.squeeze(pdbo_constrs_lists), 0)+shift, axis=0)
    lcb2_best_regret_constr_lists = np.minimum.accumulate(np.squeeze(lcb2_regret_lists) + np.maximum(np.squeeze(lcb2_constrs_lists), 0)+shift, axis=0)
    
    epbo_best_regret_constr_lists_0 = np.minimum.accumulate(np.squeeze(epbo_regret_lists_0) + np.maximum(np.squeeze(epbo_constrs_lists_0), 0)+shift, axis=0)
    
    epbo_best_regret_constr_lists_1 = np.minimum.accumulate(np.squeeze(epbo_regret_lists_1) + np.maximum(np.squeeze(epbo_constrs_lists_1), 0)+shift, axis=0)
    
    epbo_best_regret_constr_lists_1 = np.minimum.accumulate(np.squeeze(epbo_regret_lists_1) + np.maximum(np.squeeze(epbo_constrs_lists_1), 0)+shift, axis=0)

    #con_bo_best_regret_constr_lists_mean = np.mean(con_bo_best_regret_constr_lists, axis=0)
    #safe_best_regret_constr_lists_mean = np.mean(safe_best_regret_constr_lists, axis=0)
    #pdbo_best_regret_constr_lists_mean = np.mean(pdbo_best_regret_constr_lists, axis=0)
    #lcb2_best_regret_constr_lists_mean = np.mean(lcb2_best_regret_constr_lists, axis=0)
    
    #con_bo_best_regret_constr_lists_sd = np.std(con_bo_best_regret_constr_lists, axis=0)
    #safe_best_regret_constr_lists_sd = np.std(safe_best_regret_constr_lists, axis=0)
    #pdbo_best_regret_constr_lists_sd = np.std(pdbo_best_regret_constr_lists, axis=0)
    #lcb2_best_regret_constr_lists_sd = np.std(lcb2_best_regret_constr_lists, axis=0)
    
    %matplotlib inline
    #from matplotlib import rc
    #plt.rc('text',usetex=False)
   

    
    fig, axes = plt.subplots(1, 1)
    fig.set_size_inches(WIDTH * 1.0, HEIGHT * 1.0)

    fig.subplots_adjust(left=.22, bottom=.20, right=.99, top=.97, hspace=0.5, wspace=0.4) 
   
    ax = axes
    BETA1 = 0.2
    print(step_arr.shape, con_bo_best_regret_constr_lists.shape)
    plt.plot( step_arr, con_bo_best_regret_constr_lists,
                         linewidth=LINEWIDTH, color=colors[0], marker=marksCur[0], markerfacecolor='None', markersize=MARKERSIZE)
    plt.plot( step_arr, safe_best_regret_constr_lists,
                         linewidth=LINEWIDTH, color=colors[1], marker=marksCur[1], markerfacecolor='None', markersize=MARKERSIZE)
    plt.plot( step_arr, pdbo_best_regret_constr_lists, 
                         linewidth=LINEWIDTH, color=colors[2], marker=marksCur[2], markerfacecolor='None', markersize=MARKERSIZE)
    plt.plot( step_arr, epbo_best_regret_constr_lists_0,
                         linewidth=LINEWIDTH, color=colors[4], marker=marksCur[4], markerfacecolor='None', markersize=MARKERSIZE)
    
    plt.plot( step_arr, epbo_best_regret_constr_lists_1,
                         linewidth=LINEWIDTH, color=colors[5], marker=marksCur[5], markerfacecolor='None', markersize=MARKERSIZE)

    plt.plot( step_arr, lcb2_best_regret_constr_lists,
                         linewidth=LINEWIDTH, color=colors[3], marker=marksCur[3], markerfacecolor='None', markersize=MARKERSIZE)

   
    
    plt.xticks(fontsize=LABELSIZE, ticks=step_arr[::2])
    plt.yticks(fontsize=LABELSIZE)
    ax.set_xlabel('Step', fontsize=LABELSIZE)
    ax.set_ylabel('Best normalized energy \n plus normalized deviation', fontsize=LABELSIZE)  

    #ax.legend(['CEI', 'Safe BO', 'Primal-Dual', 'LCB2'])
    
    ax.legend(['CEI', 'SafeOPT', 'Primal-Dual', 'EPBO 0.2', 'EPBO 1.0', 'CONFIG'], loc='lower center', bbox_to_anchor=(0.25, -0.65, 0.5, 0.5),
             ncol=3)

    
    #ax.set_xscale('log')
    #ax.set_title('The Second Contextual Variable', fontsize=LABELSIZE)
    #ax.set_ylim(0, 0.2)
    #lgd = plt.legend(['', ''], loc='upper right', bbox_to_anchor=(1.0, 1.0), ncol=1, prop={'size': LABELSIZE})
    #fig.set_size_inches(WIDTH, HEIGHT)

    #lgd = fig.legend((aver_line[0], adv_line[0]), ('Average', 'Adversarial'), 
    #                 loc='upper right', bbox_to_anchor=(0.78, 0.1), ncol=3, prop={'size': LABELSIZE})
    plt.xlim((0, 15))

    #plt.savefig('./fig/ave_adv_comp.pdf', format='pdf', bbox_extra_artists=(lgd,), bbox_inches='tight')
    plt.savefig(f'./fig/building_funcs_r_and_v.pdf', format='pdf', bbox_inches='tight')
    plt.show()  
    
plot_best_result(con_bo_regret_lists, safe_regret_lists, pdbo_regret_lists, lcb2_regret_lists,
                    con_bo_constrs_lists, safe_constrs_lists, pdbo_constrs_lists, lcb2_constrs_lists)
